### If you dont want to use Wandb, disable Wandb otherwise optional

references for WANDB
https://analyticsindiamag.com/hands-on-guide-to-weights-and-biases-wandb-with-python-implementation/

https://docs.wandb.ai/


In [1]:
!nvidia-smi

Sun Nov 23 06:39:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   55C    P0              70W / 300W |   2192MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/workspace/Approach2


In [3]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [4]:
import os
os.environ["WANDB_DISABLED"]="true"

# Install required packages

In [5]:
# !pip install transformers==4.50.3

In [1]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.50.3


### Loading the test dataset

In [7]:
# !pip install datasets evaluate

In [8]:
from datasets import load_dataset
from evaluate import load
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")
metric = load("sacrebleu")

In [9]:
import os
print(os.getcwd())

/workspace/Approach2


The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [10]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'punct_type', 'sent_written', 'sent_meant', 'gt_marathi', 'gemini_out', 'cfilt_out'],
        num_rows: 54
    })
})

In [11]:
raw_datasets["test"][0]

{'Unnamed: 0': 0,
 'punct_type': 'Comma',
 'sent_written': 'Chanting the choir raised the volume as the celebrant intoned the prayer.',
 'sent_meant': 'Chanting, the choir raised the volume as the celebrant intoned the prayer.',
 'gt_marathi': 'जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार करताच, गायनाने आवाज वाढवला.',
 'gemini_out': 'धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या गायकवृंदाने आवाज वाढवला.',
 'cfilt_out': 'उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, गायकवृंदाने आवाज वाढवला.'}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["test"])

,Unnamed: 0,punct_type,sent_written,sent_meant,gt_marathi,gemini_out,cfilt_out
0,9,Comma,No newspaper is completely unbiased in my expert opinion.,"No newspaper is completely unbiased, in my expert opinion.","माझ्या तज्ज्ञांच्या मते, कोणतेही वृत्तपत्र पूर्णपणे निःपक्षपाती नाही.","माझ्या तज्ञांच्या मते, कोणतेही वृत्तपत्र पूर्णपणे निःपक्षपाती नसते.",तज्ज्ञांच्या मते कोणतेही वृत्तपत्र पूर्णपणे निःपक्षपाती नाही.
1,8,Comma,"X is an effective acute, oral treatment for migraine with a rapid onset of action","X is an effective acute, oral treatment for migraine, with a rapid onset of action","एक्स हा अर्धशिशीसाठी एक प्रभावी तीव्र, तोंडी उपचार आहे, ज्याची कृती जलद गतीने सुरू होते.","एक्स हे मायग्रेनसाठी एक प्रभावी तीव्र, तोंडी उपचार आहे, ज्याची क्रिया जलद सुरु होते.","एक्स हा अर्धशिशीसाठी एक प्रभावी तीव्र, तोंडी उपचार आहे, ज्यात कृतीची जलद सुरुवात होते."
2,49,Quotes,One should read G. V. Carey’s chapter Proof Correction in Mind the Stop.,One should read G. V. Carey’s chapter ‘Proof Correction’ in Mind the Stop.,जी. व्ही. कॅरी यांचा'प्रूफ करेक्शन'हा अध्याय'माईंड द स्टॉप'मध्ये वाचला पाहिजे.,जी. व्ही. कॅरीचा 'माईंड द स्टॉप' मधील 'प्रूफ करेक्शन' हा अध्याय वाचला पाहिजे.,माइंड द स्टॉप मधील जी. व्ही. कॅरी यांचा 'प्रूफ करेक्शन' हा अध्याय वाचला पाहिजे.
3,43,Comma,"This produces hard copy that can be retained but printed sheets are bulky to handle, so digital storage and viewing are often preferred.","This produces hard copy that can be retained, but printed sheets are bulky to handle, so digital storage and viewing are often preferred.","यामुळे हार्ड कॉपी तयार होते जी राखून ठेवली जाऊ शकते परंतु छापील पत्रके हाताळण्यासाठी अवजड असतात, त्यामुळे डिजिटल संचयन आणि पाहणे यांना अनेकदा प्राधान्य दिले जाते.","यामुळे जतन करता येणारी हार्ड कॉपी तयार होते, पण छापलेली पाने हाताळण्यास जाडजूड असल्याने, डिजिटल साठवण आणि पाहण्याला अनेकदा प्राधान्य दिले जाते.","यामुळे हार्ड कॉपी तयार होते जी राखून ठेवता येते, परंतु मुद्रित पत्रके हाताळण्यासाठी अवजड असतात, म्हणून डिजिटल स्टोरेज आणि पाहण्याला अनेकदा प्राधान्य दिले जाते."
4,0,Comma,Chanting the choir raised the volume as the celebrant intoned the prayer.,"Chanting, the choir raised the volume as the celebrant intoned the prayer.","जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार करताच, गायनाने आवाज वाढवला.","धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या गायकवृंदाने आवाज वाढवला.","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, गायकवृंदाने आवाज वाढवला."


In [2]:
def initialize_model(ckpt_dir, device, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to(device)
        if DEVICE == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return model

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

en_indic_ckpt_dir = "thenlpresearcher/iitb-en-indic-without-punct"
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

# "ai4bharat/indictrans2-indic-indic-dist-320M"

device = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)

In [4]:
def initialize_model(ckpt_dir, device, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to_empty(device=device)
        if device == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return model

In [5]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained("thenlpresearcher/iitb_en_indic_robust_punctuation_model", trust_remote_code=True)
en_indic_model = initialize_model(model_checkpoint, device)

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [30]:
tokenizer = en_indic_tokenizer

In [31]:
len(tokenizer)

32322

In [32]:
tokenizer

IndicTransTokenizer(name_or_path='ai4bharat/indictrans2-en-indic-dist-200M', vocab_size=32322, model_max_length=256, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [33]:
raw_datasets['test']

Dataset({
    features: ['Unnamed: 0', 'punct_type', 'sent_written', 'sent_meant', 'gt_marathi', 'gemini_out', 'cfilt_out'],
    num_rows: 54
})

In [6]:
from IndicTransToolkit.processor import IndicProcessor

In [7]:
ip = IndicProcessor(inference=True)

In [22]:
BATCH_SIZE = 4
src_lang, tgt_lang = "eng_Latn", "mar_Deva"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [23]:
print(len(src_sentences))
print(len(ref_gt))
print(len(ref_gem))
print(len(ref_cfilt))

54
54
54
54


In [24]:
print(ip.preprocess_batch(["Chanting the choir raised…"], src_lang="eng_Latn", tgt_lang="mar_Deva"))

['eng_Latn mar_Deva Chanting the choir raised ...']


In [67]:
import torch

def debug_translate_one(
    sentence: str,
    src_lang: str,
    tgt_lang: str,
    model,
    tokenizer,
    ip,
    device: str = "cuda",
    max_length: int = 256,
):
    """
    Translate a single sentence using IndicTrans2 with detailed debug output.

    Args:
        sentence (str): Input sentence to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model (IndicTrans2).
        tokenizer: Corresponding tokenizer.
        ip: IndicProcessor object for preprocessing/postprocessing.
        device (str): Torch device ("cuda" or "cpu").
        max_length (int): Maximum length of generated sequence.

    Returns:
        str: Postprocessed translation.
    """
    model.to(device)
    model.eval()

    print("\n=== INPUT SENTENCE ===")
    print(sentence)

    # ------------------------ Preprocessing ------------------------
    preprocessed = ip.preprocess_batch([sentence], src_lang=src_lang, tgt_lang=tgt_lang)[0]
    print("\n=== PREPROCESSED ===")
    print(preprocessed)

    # ------------------------ Tokenization ------------------------
    inputs = tokenizer(
        [preprocessed],
        return_tensors="pt",
        padding="longest",
        truncation=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    print("\n=== TOKEN IDS (first 30) ===")
    print(inputs["input_ids"][0][:30])

    print("\n=== TOKENIZED INPUT TOKENS ===")
    print(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][:30]))

    # ------------------------ Decoder token fix ------------------------
    if model.config.decoder_start_token_id is None:
        fixed_id = tokenizer.convert_tokens_to_ids(tgt_lang)
        model.config.decoder_start_token_id = fixed_id
        print(f"[Fix] decoder_start_token_id set to: {fixed_id}")
    else:
        fixed_id = model.config.decoder_start_token_id
    
    print(f"Fixed id: {fixed_id}")
    pad_token_id = tokenizer.pad_token_id
    eos_token_id = tokenizer.eos_token_id

    # ------------------------ Generation ------------------------
    print("\n=== GENERATING... ===")
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            max_length=max_length,
            min_length=5,            # ensure meaningful output
            num_beams=5,
            num_return_sequences=1,
            early_stopping=True,
            decoder_start_token_id=fixed_id,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            use_cache=True,
        )
        
    print("\n=== GENERATED TOKEN IDS ===")
    print(generated_tokens)

    # ------------------------ Decode ------------------------
    decoded_text = tokenizer.decode(
        generated_tokens[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    print("\n=== DECODED RAW ===")
    print(decoded_text)

    # ------------------------ Postprocess ------------------------
    try:
        post = ip.postprocess_batch([decoded_text], lang=tgt_lang)[0]
    except Exception as e:
        print(f"[Warning] Postprocess failed: {e}")
        post = decoded_text

    print("\n=== FINAL POSTPROCESSED TRANSLATION ===")
    print(post)

    return post

In [12]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [13]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [14]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # ai4bharat/indictrans2-en-indic-dist-200M
# "ai4bharat/indictrans2-indic-indic-dist-320M"
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
    "Let’s eat, Grandma."
]

# en_sents = sentences_meant

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

# df['gt -- marathi'] = hi_translations


eng_Latn - mar_Deva
eng_Latn: When I was young, I used to go to the park every day.
mar_Deva: मी लहान होतो तेव्हा मी दररोज उद्यानात जायचो.
eng_Latn: He has many old books, which he inherited from his ancestors.
mar_Deva: त्यांच्याकडे अनेक जुनी पुस्तके आहेत, जी त्यांना त्यांच्या पूर्वजांकडून वारशाने मिळाली आहेत.
eng_Latn: I can't figure out how to solve my problem.
mar_Deva: माझी समस्या कशी सोडवायची हे मला समजत नाही.
eng_Latn: She is very hardworking and intelligent, which is why she got all the good marks.
mar_Deva: ती खूप मेहनती आणि हुशार आहे, म्हणूनच तिला सर्व चांगले गुण मिळाले.
eng_Latn: We watched a new movie last week, which was very inspiring.
mar_Deva: आम्ही गेल्या आठवड्यात एक नवीन चित्रपट पाहिला, जो खूप प्रेरणादायी होता.
eng_Latn: If you had met me at that time, we would have gone out to eat.
mar_Deva: त्यावेळी तुम्ही मला भेटलात तर आम्ही जेवायला बाहेर गेलो असतो.
eng_Latn: She went to the market with her sister to buy a new sari.
mar_Deva: ती तिच्या बहिणीसोबत नवीन साडी विकत घेण

In [24]:
def initialize_model(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(device=DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return model

In [16]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # ai4bharat/indictrans2-en-indic-dist-200M
# "ai4bharat/indictrans2-indic-indic-dist-320M"
en_indic_model = initialize_model(en_indic_ckpt_dir, quantization)
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

en_indic_ckpt_dir = "thenlpresearcher/iitb-en-indic-without-punct"
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

# "ai4bharat/indictrans2-indic-indic-dist-320M"

device = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)


ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
    "Let’s eat, Grandma."
]

# en_sents = sentences_meant

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

# df['gt -- marathi'] = hi_translations


eng_Latn - mar_Deva
eng_Latn: When I was young, I used to go to the park every day.
mar_Deva: मी लहान होतो तेव्हा मी दररोज उद्यानात जायचो.
eng_Latn: He has many old books, which he inherited from his ancestors.
mar_Deva: त्यांच्याकडे अनेक जुनी पुस्तके आहेत, जी त्यांना त्यांच्या पूर्वजांकडून वारशाने मिळाली आहेत.
eng_Latn: I can't figure out how to solve my problem.
mar_Deva: माझी समस्या कशी सोडवायची हे मला समजत नाही.
eng_Latn: She is very hardworking and intelligent, which is why she got all the good marks.
mar_Deva: ती खूप मेहनती आणि हुशार आहे, म्हणूनच तिला सर्व चांगले गुण मिळाले.
eng_Latn: We watched a new movie last week, which was very inspiring.
mar_Deva: आम्ही गेल्या आठवड्यात एक नवीन चित्रपट पाहिला, जो खूप प्रेरणादायी होता.
eng_Latn: If you had met me at that time, we would have gone out to eat.
mar_Deva: त्यावेळी तुम्ही मला भेटलात तर आम्ही जेवायला बाहेर गेलो असतो.
eng_Latn: She went to the market with her sister to buy a new sari.
mar_Deva: ती तिच्या बहिणीसोबत नवीन साडी विकत घेण

In [26]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cpu"
quantization = None

In [28]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_ckpt_dir = "thenlpresearcher/iitb-en-indic-without-punct"
from accelerate import init_empty_weights

# "ai4bharat/indictrans2-indic-indic-dist-320M"
en_indic_model = initialize_model(en_indic_ckpt_dir, quantization)
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

# "ai4bharat/indictrans2-indic-indic-dist-320M"

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
quantization = None
en_indic_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)


ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
    "Let’s eat, Grandma."
]

# en_sents = sentences_meant

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

# df['gt -- marathi'] = hi_translations

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [44]:
print(tokenizer.convert_tokens_to_ids("eng_Latn"))
print(tokenizer.convert_tokens_to_ids("mar_Deva"))

4
29


In [25]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans with safety checks.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    # Safe access for decoder_start_token_id
    decoder_start_token_id = getattr(model.config, "decoder_start_token_id", None)
    pad_token_id = getattr(tokenizer, "pad_token_id", None)
    eos_token_id = getattr(tokenizer, "eos_token_id", None)

    if decoder_start_token_id is None:
        print("[Warning] decoder_start_token_id is None. Using default generation behavior.")

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]
        print('here')

        # Preprocess the batch
        batch_preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        if not isinstance(batch_preprocessed, list) or len(batch_preprocessed) == 0:
            print(f"[Warning] Preprocessed batch is empty at index {i}. Skipping...")
            continue

#         Debug: print first 2 sentences after preprocessing
        print(f"[Debug] Preprocessed batch sample: {batch_preprocessed[:2]}")

        # Tokenize
        inputs = tokenizer(
            batch_preprocessed,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations with safety parameters
        with torch.no_grad():
            try:
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=5,
                    max_length=max_length,
                    num_beams=5,
                    num_return_sequences=1,
                    early_stopping=True,
                    decoder_start_token_id=decoder_start_token_id,
                    pad_token_id=pad_token_id,
                    eos_token_id=eos_token_id
                )
            except Exception as e:
                print(f"[Error] Generation failed for batch starting at index {i}: {e}")
                continue

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

#         Debug: print first 2 decoded outputs
        print(f"[Debug] Decoded sample: {decoded_texts[:2]}")

        # Postprocess translations
        try:
            postprocessed = ip.postprocess_batch(decoded_texts, lang=tgt_lang)
            translations += postprocessed
        except Exception as e:
            print(f"[Error] Postprocessing failed for batch starting at index {i}: {e}")
            translations += decoded_texts  # fallback

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [26]:
src_lang, tgt_lang = "eng_Latn", "mar_Deva"

prefix = f"{tgt_lang} {src_lang}"

def remove_prefix(text):
    if text.startswith(prefix):
        return text[len(prefix):].strip()
    return text.strip()

In [36]:
from evaluate import load
from tqdm import tqdm
import pandas as pd

# -------------------- TRANSLATION --------------------
valid_src = []
valid_pred = []
valid_gt = []
valid_gem = []
valid_cfilt = []

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
#     print(batch)
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        en_indic_model,
        en_indic_tokenizer,
        ip,
        device=device
    )
    all_translations.extend(translations)

    if translations is None:
        print(f"[SKIPPED] Batch {i}: Returned None")
        continue

    cleaned = [remove_prefix(t) for t in translations]

    valid_src.extend(batch)
    valid_pred.extend(cleaned)
    valid_gt.extend(ref_gt[i:i + len(batch)])
    valid_gem.extend(ref_gem[i:i + len(batch)])
    valid_cfilt.extend(ref_cfilt[i:i + len(batch)])

print(f"\nSuccessful translations: {len(valid_pred)} / {len(src_sentences)}")

 14%|█████▋                                  | 2/14 [00:00<00:01,  9.57it/s]

here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Chanting the choir raised the volume as the celebrant intoned the prayer .', 'eng_Latn mar_Deva A six-month-old calf was submitted for examination , showing lameness in all four legs which had been present since soon after birth .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva As mentioned , first impressions can be misleading .', 'eng_Latn mar_Deva To get a clean assembly load the assembled equals table before the assembly is run .']
[Debug] Decoded sample: ['"', '"']


 21%|████████▌                               | 3/14 [00:00<00:01,  8.96it/s]

here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva X is an effective acute , oral treatment for migraine with a rapid onset of action', 'eng_Latn mar_Deva No newspaper is completely unbiased in my expert opinion .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva The only problem with the new project established in the desert at high cost is the lack of good access roads .', 'eng_Latn mar_Deva Having failed in all previous attempts he evolved a new plan which surprised everybody .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva In his daily traffickings a Cairene resident is often made conscious of the suffocating pollution .', 'eng_Latn mar_Deva If drama implies conflict and poetry metaphor then poetic drama must imply the dramatization of metaphor !']


 50%|████████████████████                    | 7/14 [00:00<00:00, 14.74it/s]

[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva The calculator executed the functions that had been selected and displayed the results on the top half of the screen .', 'eng_Latn mar_Deva There are many disturbing factors fatigue , poor eye-sight , poor reading ability , anxiety or undue caution , distractibility , and inadequate motivation .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva The crucial words in the specification are the signals must be routed internally .', 'eng_Latn mar_Deva Would you please supply a list of the correct settings for the ABC']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva However oxidative stress occurs when the balance is disturbed , through an increase in reactive oxygen species', "eng_Latn mar_Deva Should any burner fail to ignite its respective section will revert to ' purge ' and in this way the system ens

 79%|██████████████████████████████▋        | 11/14 [00:00<00:00, 16.86it/s]

[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ["eng_Latn mar_Deva Insert the new disk into the disk drive with the notch at the bottom facing the drive 's entry slot .", 'eng_Latn mar_Deva Replace the fuel lines and electrical conduits , which have cracks or damaged B-nut fittings to prevent leaks and ensure safe operation .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Take no action as the camera operates automatically under all lighting conditions .', 'eng_Latn mar_Deva The species of fish supported by the reef are varied and abundant food supplies are available to sustain their populations .']
[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Connection to PTT-supplied packet-switch networks will be a prime requirement of the workstation and gateways into these networks are planned by the PTTs to ensure seamless data communication .', 'eng_Latn mar_Deva The s

100%|███████████████████████████████████████| 14/14 [00:00<00:00, 15.20it/s]

[Debug] Decoded sample: ['"', '"']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva a water : glycol ratio of < ID1 > is preferred .', 'eng_Latn mar_Deva It has thickness of only < ID1 > inch .']
[Debug] Decoded sample: ['"', '"']

Successful translations: 54 / 54


In [ ]:
valid_pred[2]

In [29]:
mode = "without"

In [30]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": valid_src,
    "prediction": valid_pred,
    "gt": valid_gt,
    "gemini": valid_gem,
    "cfilt": valid_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(valid_pred, valid_gt, valid_gem, valid_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gt])["score"],
    "Gemini": bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gem])["score"],
    "CFILT":  bleu.compute(predictions=valid_pred, references=[[r] for r in valid_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to combined_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 0.03, chrF++: 0.01
GT Marathi → BLEU: 0.02
Gemini    → BLEU: 0.02
CFILT     → BLEU: 0.02

🎯 BEST REFERENCE = Gemini (by highest BLEU)
Metrics written to punct_combined_baseline_outputs_eval_metrics.txt


In [33]:
results_df['prediction'][0]

'उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने आवाज वाढला.'